# Pricing simulation with Q-learning agents in Julia

This is a Julia implementation of a simulation study in which Q-learning agents compete in a market environment with
inelastic demand.


# The market environment
The market environment is described in detail in the paper ["Algorithmic and Human Collusion"](https://tofewe.github.io/Algorithmic_and_Human_Collusion_Tobias_Werner.pdf).

In short: 
- Price competition with homogeneous goods and no production costs in a repeated game
- Market consists of $N\in \{2,3\}$ firms 
- Firms are represented by a Q-learning algorithm
- 60 computerized consumers with perfectly inelastic unit demand and a reservation price of $p^{max}=4$
- In each period $t$, firms set prices simultaneously $p_i^t\in \mathscr{P} := \{0,... , 5\}$
- Market is shared in each round among the firms with the lowest price

However, the environment can be customized easily (see below).

The profit function `calc_reward` computes the profits in each period.

# Defining the parameters of the simulation
The DataType `SimParams` from the `sim_types.jl` module defines the main parameters that describe the simulation setup. See below for a documentation.

In [10]:
include("sim_types.jl")

SimParams

In [11]:
?SimParams

search: SimParams



The parameters required for the simulation.

# Fields:

  * n_firms: Number of firms in the market
  * min_price: Minimal price
  * max_price: Maximal price
  * reservation_price: reservation price of the consumers
  * m_consumer: Number of consumers in the market
  * k_memory: Memory of the agent
  * beta: Exploration decay of the agent
  * discount_rate: Discount rate as in any economic model
  * alpha: Learning rate
  * max_t: Maximal number of learning iterations
  * rounds_convergence: Number of rounds to determine convergence
  * optimality_threshold: Threshold to determine convergence for q*


We use the parameters from the paper but it can be easily adjusted.


In [12]:
n_firms = 2
min_price = 0
max_price = 5
reservation_price = 4
m_consumer = 60
k_memory = 1
beta = 4e-07
discount_rate = 0.95
alpha = 0.04
max_t = 1000000000
rounds_convergence = 100000
optimality_threshold = 0.00000000001

current_params = SimParams(n_firms = n_firms,
                           min_price = min_price,
                           max_price = max_price,
                           reservation_price = reservation_price,
                           m_consumer = m_consumer,
                           k_memory = k_memory,
                           beta = beta,
                           discount_rate = discount_rate,
                           alpha = alpha,
                           max_t = max_t,
                           rounds_convergence = rounds_convergence,
                           optimality_threshold=optimality_threshold)

SimParams(2, 0, 5, 4, 60, 1, 4.0e-7, 0.95, 0.04, 1000000000, 100000, 1.0e-11)

In [13]:
include("market_env.jl")

calc_reward

In [14]:
?calc_reward

search: calc_reward



Return the profit for the firm in the market.

# Arguments:

  * `price::Integer`: Price of the firm
  * `all_prices::Array{Integer,1}`: All prices in the market in the given round
  * `p::SimParams`: Market parameter


# Q-learning agents in the market environment

We have to make some assumption how algorithms perceive the market environment:
- The action set given by the set of possible prices $\mathbf{A} \equiv \mathscr{P}$
    - Makes inutitive sense since we look at pricing algorithms
- State $s_t$ is the combination of past prices $s_t = \{p^{t-1}_1, ..., p^{t-1}_N\}$
    - Boils down to memory-one strategies predominantly used by humans in PD
- Reward signal is given by the economic profit $\pi^t_i$

The main function are implemented in the `agent.jl` module.

In [15]:
include("agent.jl")

calc_epsilon

# Training simulations


In [16]:
include("utils_simulation.jl")


create_transition_dict

In [17]:
function train_agent(seed::Int64, p::SimParams)
    # Create local random number generator
    local_RNG = MersenneTwister(seed)
    
    # Create variable from the parameters p
    all_prices, all_price_states, n_states, n_actions = create_vars(p)

    # Create mapping dicts from int states (column index) to prices and vice versa
    int_to_price_state, price_state_to_int = create_transition_dict(all_price_states, n_states)
    # Create Q-matrices
    all_q = [rand(local_RNG, Float64, n_states, n_actions) for _ in 1:p.n_firms]
    
    # Start with the training
    # Arbitraty start stae
    state = 1
    next_state = 1
    # Init convergence counter 
    convergence_count = 0
    
    # Init best action
    best_actions = []
    
    
    for t in 1:p.max_t
        epsilon = calc_epsilon(t, beta)
        best_actions_old = copy(best_actions)
        current_actions = tuple([get_action(q, state, epsilon, n_actions, local_RNG) for q in all_q]...)
        current_prices = collect(index_to_price(current_actions)) #Collect to get an array
        current_profits = [calc_reward(price, current_prices, p) for price in current_prices]
        next_state = price_state_to_int[tuple(current_prices...)] #Cast to tuple again due to hash

        # Get best actions before the update
        best_actions_before_update = [get_best_action(q, state) for q in all_q]

        # Update
        all_q = [update(q, state, action, reward, next_state, p) for (q, action, reward) in zip(all_q, current_actions, current_profits)]

        # Get best actions after the update, Note that only the best action for the given 
        # state could have changed
        best_action_after_update = [get_best_action(q, state) for q in all_q]

        # check convergence
        if best_actions_before_update == best_action_after_update
            convergence_count += 1
            if convergence_count > p.rounds_convergence
                break
            end
        else
            convergence_count = 0
        end
        # State is the new state from the last period
        state = next_state
    end    
    return state, all_q
end

train_agent (generic function with 1 method)

In [18]:
@time convergence_state, all_q =  train_agent(6, current_params)


 34.450283 seconds (335.78 M allocations: 24.849 GiB, 11.29% gc time)


(29, [[1648.8671342423718 2080.8463277747783 … 1731.3639410891965 1732.9709092358225; 1919.2879404227992 1923.8881296448887 … 2031.3187115020885 1903.442959982311; … ; 2011.0488630223022 2020.9026847747546 … 2035.9271157486503 1927.7129222726355; 1719.95837213905 2137.8690134186572 … 1865.750459160043 1753.2403060634565], [1656.935474509337 1670.895865518859 … 1639.39630813921 1687.3168718941777; 1885.1706516508577 1927.5296435923344 … 1840.7675616280355 1895.7199627028772; … ; 2052.0615171194927 2210.0951786637816 … 2137.4965740441394 2059.0308552001866; 1705.3580438008694 1694.3219286948456 … 2163.721003912153 1682.1541203531033]])